Language composition by repository

This report shows the different languages that are used in each repository.

In [ ]:
unit: str = "files"

In [ ]:
import plotly.express as px
from code_data_science import data_table as dt
import code_data_science.palette as palette

df = dt.read_csv("../samples/language_composition.csv")
df["files"] = 1

# Exit early if there are no rows and render a plot with a message
if len(df) == 0:
    fig = px.treemap(names=[], parents=[])
    fig.add_annotation(
        x=0.5, y=0.5, text="No rows of data found", showarrow=False, font=dict(size=20)
    )
else:
    # if there is no sourceFileType column and there is a parserClass column then rename the parserClass column to sourceFileType
    if "sourceFileType" not in df.columns and "parserClass" in df.columns:
        df.rename(columns={"parserClass": "sourceFileType"}, inplace=True)

    # concat origin:path:branch into repository
    df["repository"] = (
        df["repositoryOrigin"]
        + ":"
        + df["repositoryPath"]
        + ":"
        + df["repositoryBranch"]
    )
    df["fileType"] = df["sourcePath"].str.split(".").str[-1]
    df_file_grouped = df.groupby(
        by=["repository", "language", "fileType", "sourceFileType", "hasParseFailures"],
        as_index=False,
    ).sum(numeric_only=True)

    path = ["repository", "language", "fileType"]

    # number of rows in the dataframe
    total_files = len(df["sourcePath"])

    # number of rows with a language of Other/unknown/unparseable
    total_unparseable_files = len(df[df["language"] == "Other/unknown/unparseable"])

    # number of rows with a file type of java but a language of Other/unknown/unparseable
    total_java_unparseable_files = len(
        df[(df["language"] == "Other/unknown/unparseable") & (df["fileType"] == "java")]
    )

    # number of rows with parse failures
    total_parse_failures = len(df[df["hasParseFailures"] == True])

    # number of rows with a file type of java and parse failures
    total_java_parse_failures = len(
        df[(df["hasParseFailures"] == True) & (df["fileType"] == "java")]
    )

    # determine measurement field based on `unit`
    measurement_field = "files" if unit == "files" else "linesOfText"

    # sum of lines of text in java files affected by parse failures
    total_lines_of_java_affected_by_parse_failures = df[
        (df["hasParseFailures"] == True) & (df["fileType"] == "java")
    ][measurement_field].sum()

    # Set the color palette
    colors = palette.qualitative()

    fig = px.treemap(
        df_file_grouped,
        path=path,
        values=measurement_field,
        color_discrete_sequence=colors,
    )

    fig.update_layout(
        margin=dict(t=28, l=0, r=0, b=0),
        annotations=[
            dict(
                x=0.01,
                y=1.055,
                showarrow=False,
                borderwidth=1,
                text="<b>More info</b>",
                hovertext=(
                    f"""
            <b>Total files:</b> {"{:,}".format(total_files)} </br> </br>
            <b>Other/unknown/unparseable files:</b> {"{:,}".format(total_unparseable_files)}</br>
            <b>Other/unknown/unparseable .java files:</b> {"{:,}".format(total_java_unparseable_files)} </br>
            <b>Files with parse failures:</b> {"{:,}".format(total_parse_failures)} </br>
            <b>.java files with parse failures:</b> {"{:,}".format(total_java_parse_failures)} </br>
            <b>Total lines of text in .java files with parse failures:</b> {"{:,}".format(total_lines_of_java_affected_by_parse_failures)}
            """
                ),
                xref="paper",
                yref="paper",
            )
        ],
    )

    fig.update_traces(
        marker=dict(cornerradius=3),
    )

    fig.data[0].textinfo = "label+text+value"

In [ ]:
# Output the visualization
fig.show(render="plotly_mimetype")